In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/2025-citibike.csv', dtype='str', index_col=[0])
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,56BD148A05E26915,electric_bike,2025-01-01 22:19:06.324,2025-01-01 22:23:32.899,W 36 St & 7 Ave,6483.06,W 24 St & 7 Ave,6257.03,40.752149,-73.989539,40.74487634,-73.99529885,member
1,700CCCDF00C08077,electric_bike,2025-01-09 15:20:43.991,2025-01-09 15:26:19.137,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.73829,-73.99006,40.74487634,-73.99529885,member
2,B1A3FAFA5FE651CE,electric_bike,2025-01-01 12:54:26.571,2025-01-01 13:03:03.565,Jackson Ave & 46 Rd,6203.02,Crescent St & Broadway,6827.11,40.74524768,-73.94733276,40.7633589,-73.9286471,member
3,9CB0F28054BA5EBC,electric_bike,2025-01-14 13:43:59.151,2025-01-14 13:49:04.543,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.73829,-73.99006,40.74487634,-73.99529885,member
4,A1B8FDD4FAF0C2B2,electric_bike,2025-01-13 07:20:37.176,2025-01-13 07:29:18.015,Willis Ave & E 143 St,7798.02,1 Ave & E 110 St,7522.02,40.812299,-73.92037,40.7923272,-73.9383,member


In [3]:
df.describe()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
count,15374152,15374152,15374152,15374152,15368777,15368777,15336066,15334169,15374152,15374152,15371547,15371547,15374152
unique,15374152,2,15356511,15356490,2167,2260,2233,2304,2401,2391,2255,2248,2
top,56BD148A05E26915,electric_bike,2025-04-17 15:45:48.098,2025-03-18 10:57:16.737,W 21 St & 6 Ave,6140.05,W 21 St & 6 Ave,6140.05,40.74173969,-73.99415556,40.74173969,-73.99415556,member
freq,1,10735496,3,19,58261,58261,58394,58393,58261,58261,58393,58393,13074824
